# MultiOmicsNet: Complete Workflow Example

This notebook demonstrates the complete workflow for multi-omics cross-layer network integration using the MultiOmicsNet algorithm.

## Overview

We will integrate:
- Single-cell RNA-seq data
- Single-cell ATAC-seq data  
- Bulk metabolomics data
- 16S rRNA microbiome data

And perform differential network analysis to identify changes across conditions (e.g., aging, disease, treatment).

## 1. Setup and Data Loading

In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import scanpy as sc
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Import MultiOmicsNet
import multiomicsnet as mon

# Set up scanpy settings
sc.settings.verbosity = 3
sc.settings.set_figure_params(dpi=80, facecolor='white')

print("MultiOmicsNet version:", mon.__version__)

### Load Example Data

For this example, we'll use simulated data that represents a typical multi-omics experiment comparing young vs aged samples.

In [ ]:
# Generate example data (in practice, you would load your own data)
np.random.seed(42)

# Simulate single-cell RNA-seq data
n_genes = 2000
n_cells = 1000
rna_data = pd.DataFrame(
    np.random.negative_binomial(10, 0.3, (n_genes, n_cells)),
    index=[f"Gene_{i}" for i in range(n_genes)],
    columns=[f"Cell_{i}" for i in range(n_cells)]
)

# Create cell metadata
rna_metadata = pd.DataFrame({
    'cell_id': [f"Cell_{i}" for i in range(n_cells)],
    'condition': ['young'] * 500 + ['aged'] * 500,
    'cell_type': np.random.choice(['T_cell', 'B_cell', 'Monocyte', 'NK_cell'], n_cells),
    'batch': np.random.choice(['batch1', 'batch2'], n_cells)
})

print(f"RNA data shape: {rna_data.shape}")
print(f"Conditions: {rna_metadata['condition'].value_counts()}")

In [ ]:
# Simulate single-cell ATAC-seq data
n_peaks = 5000
n_cells_atac = 800  # Slightly fewer cells for ATAC
atac_data = pd.DataFrame(
    np.random.binomial(1, 0.1, (n_peaks, n_cells_atac)),
    index=[f"Peak_{i}" for i in range(n_peaks)],
    columns=[f"Cell_{i}" for i in range(n_cells_atac)]
)

# Create peak metadata with genomic coordinates
peak_metadata = pd.DataFrame({
    'peak_id': [f"Peak_{i}" for i in range(n_peaks)],
    'chr': np.random.choice([f'chr{i}' for i in range(1, 23)], n_peaks),
    'start': np.random.randint(1000, 100000, n_peaks),
    'end': lambda x: x + np.random.randint(200, 2000, n_peaks)
})
peak_metadata['end'] = peak_metadata['start'] + np.random.randint(200, 2000, n_peaks)

# ATAC cell metadata
atac_metadata = pd.DataFrame({
    'cell_id': [f"Cell_{i}" for i in range(n_cells_atac)],
    'condition': ['young'] * 400 + ['aged'] * 400,
    'cell_type': np.random.choice(['T_cell', 'B_cell', 'Monocyte', 'NK_cell'], n_cells_atac),
    'batch': np.random.choice(['batch1', 'batch2'], n_cells_atac)
})

print(f"ATAC data shape: {atac_data.shape}")

In [ ]:
# Simulate bulk metabolomics data
n_metabolites = 200
n_samples_metab = 40
metabolomics_data = pd.DataFrame(
    np.random.lognormal(0, 1, (n_metabolites, n_samples_metab)),
    index=[f"Metabolite_{i}" for i in range(n_metabolites)],
    columns=[f"Sample_{i}" for i in range(n_samples_metab)]
)

# Metabolomics sample metadata
metabolomics_metadata = pd.DataFrame({
    'sample_id': [f"Sample_{i}" for i in range(n_samples_metab)],
    'condition': ['young'] * 20 + ['aged'] * 20,
    'batch': np.random.choice(['batch1', 'batch2'], n_samples_metab)
})

# Metabolite annotations
metabolite_metadata = pd.DataFrame({
    'metabolite_id': [f"Metabolite_{i}" for i in range(n_metabolites)],
    'pathway': np.random.choice(['Glycolysis', 'TCA_cycle', 'Lipid_metabolism', 'Amino_acid_metabolism'], n_metabolites),
    'class': np.random.choice(['Organic_acid', 'Lipid', 'Amino_acid', 'Nucleotide'], n_metabolites)
})

print(f"Metabolomics data shape: {metabolomics_data.shape}")

In [ ]:
# Simulate 16S rRNA microbiome data
n_taxa = 150
n_samples_micro = 40
microbiome_data = pd.DataFrame(
    np.random.negative_binomial(50, 0.1, (n_taxa, n_samples_micro)),
    index=[f"Taxa_{i}" for i in range(n_taxa)],
    columns=[f"Sample_{i}" for i in range(n_samples_micro)]
)

# Microbiome sample metadata (same as metabolomics)
microbiome_metadata = metabolomics_metadata.copy()

# Taxonomy metadata
taxonomy_metadata = pd.DataFrame({
    'taxa_id': [f"Taxa_{i}" for i in range(n_taxa)],
    'phylum': np.random.choice(['Firmicutes', 'Bacteroidetes', 'Proteobacteria', 'Actinobacteria'], n_taxa),
    'genus': [f"Genus_{i}" for i in range(n_taxa)],
    'species': [f"Species_{i}" for i in range(n_taxa)]
})

print(f"Microbiome data shape: {microbiome_data.shape}")

## 2. Initialize MultiOmicsNet and Add Data

In [ ]:
# Initialize the MultiOmicsIntegrator
integrator = mon.MultiOmicsIntegrator(
    integration_method='hybrid',  # Use both inference and knowledge-based methods
    scenic_plus=True,             # Use SCENIC+ for gene regulatory networks
    scvi_integration=True,        # Use scVI for single-cell integration
    random_state=42,
    verbose=True
)

In [ ]:
# Add RNA-seq data
integrator.add_rna_data(
    data=rna_data,
    sample_metadata=rna_metadata,
    data_type='counts'
)

# Add ATAC-seq data
integrator.add_atac_data(
    data=atac_data,
    sample_metadata=atac_metadata,
    peak_metadata=peak_metadata,
    data_type='counts'
)

# Add metabolomics data
integrator.add_metabolomics_data(
    data=metabolomics_data,
    sample_metadata=metabolomics_metadata,
    metabolite_metadata=metabolite_metadata,
    data_type='abundance'
)

# Add microbiome data
integrator.add_microbiome_data(
    data=microbiome_data,
    sample_metadata=microbiome_metadata,
    taxonomy_metadata=taxonomy_metadata,
    data_type='counts'
)

## 3. Data Preprocessing

In [ ]:
# Preprocess all data types
integrator.preprocess_data(
    rna_params={
        'min_genes': 200,
        'min_cells': 3,
        'max_mito_pct': 20,
        'normalize': True,
        'log_transform': True,
        'highly_variable': True,
        'n_top_genes': 2000
    },
    atac_params={
        'min_peaks': 500,
        'min_cells': 3,
        'normalize': True,
        'binary_transform': True
    },
    metabolomics_params={
        'log_transform': True,
        'scale': True,
        'impute_missing': True
    },
    microbiome_params={
        'min_prevalence': 0.1,
        'clr_transform': True,
        'filter_rare': True
    }
)

## 4. Network Construction

In [ ]:
# Build cross-layer networks
networks = integrator.build_networks(
    method='hybrid',
    network_params={
        'correlation_threshold': 0.3,
        'significance_threshold': 0.05,
        'use_scenic_plus': True,
        'use_knowledge_db': True,
        'n_neighbors': 15
    }
)

print("Available networks:", list(networks.keys()))

## 5. Network Quantification

In [ ]:
# Compute network metrics
metrics = integrator.compute_network_metrics(
    metrics_list=[
        'degree_centrality',
        'betweenness_centrality',
        'closeness_centrality',
        'clustering_coefficient',
        'modularity',
        'small_worldness',
        'network_density'
    ]
)

print("Computed metrics:", list(metrics.keys()))

In [ ]:
# Display network summary statistics
if 'network_summary' in metrics:
    print("Network Summary Statistics:")
    print(metrics['network_summary'])

## 6. Differential Network Analysis

In [ ]:
# Perform differential network analysis between young and aged conditions
diff_results = integrator.differential_analysis(
    condition_column='condition',
    control='young',
    treatment='aged',
    method='permutation',
    n_permutations=1000,
    alpha=0.05
)

print("Differential analysis results:", list(diff_results.keys()))

In [ ]:
# Display significant differential edges
if 'differential_edges' in diff_results:
    sig_edges = diff_results['differential_edges']
    sig_edges_filtered = sig_edges[sig_edges['p_value'] < 0.05]
    
    print(f"Number of significant differential edges: {len(sig_edges_filtered)}")
    print("\nTop 10 most significant differential edges:")
    print(sig_edges_filtered.head(10))

## 7. Visualization

In [ ]:
# Plot network overview
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Plot degree distribution
if 'degree_centrality' in metrics:
    degree_values = list(metrics['degree_centrality'].values())
    axes[0, 0].hist(degree_values, bins=30, alpha=0.7)
    axes[0, 0].set_title('Degree Distribution')
    axes[0, 0].set_xlabel('Degree')
    axes[0, 0].set_ylabel('Frequency')

# Plot clustering coefficient distribution
if 'clustering_coefficient' in metrics:
    clustering_values = list(metrics['clustering_coefficient'].values())
    axes[0, 1].hist(clustering_values, bins=30, alpha=0.7, color='orange')
    axes[0, 1].set_title('Clustering Coefficient Distribution')
    axes[0, 1].set_xlabel('Clustering Coefficient')
    axes[0, 1].set_ylabel('Frequency')

# Plot network metrics comparison
if 'network_summary' in metrics:
    summary_df = metrics['network_summary']
    if len(summary_df) > 1:
        summary_df.plot(kind='bar', ax=axes[1, 0])
        axes[1, 0].set_title('Network Metrics Comparison')
        axes[1, 0].tick_params(axis='x', rotation=45)

# Plot differential analysis results
if 'differential_edges' in diff_results:
    p_values = diff_results['differential_edges']['p_value']
    axes[1, 1].hist(-np.log10(p_values), bins=30, alpha=0.7, color='red')
    axes[1, 1].axvline(-np.log10(0.05), color='black', linestyle='--', label='p=0.05')
    axes[1, 1].set_title('Differential Analysis P-values')
    axes[1, 1].set_xlabel('-log10(p-value)')
    axes[1, 1].set_ylabel('Frequency')
    axes[1, 1].legend()

plt.tight_layout()
plt.show()

In [ ]:
# Create a volcano plot for differential edges
if 'differential_edges' in diff_results:
    diff_edges = diff_results['differential_edges']
    
    plt.figure(figsize=(10, 8))
    
    # Calculate -log10(p-value) and effect size
    neg_log_p = -np.log10(diff_edges['p_value'])
    effect_size = diff_edges.get('effect_size', np.random.normal(0, 1, len(diff_edges)))
    
    # Color points based on significance
    colors = ['red' if p < 0.05 else 'gray' for p in diff_edges['p_value']]
    
    plt.scatter(effect_size, neg_log_p, c=colors, alpha=0.6)
    plt.axhline(-np.log10(0.05), color='black', linestyle='--', alpha=0.5)
    plt.xlabel('Effect Size')
    plt.ylabel('-log10(p-value)')
    plt.title('Volcano Plot: Differential Network Edges (Young vs Aged)')
    
    # Add legend
    from matplotlib.patches import Patch
    legend_elements = [Patch(facecolor='red', label='Significant (p < 0.05)'),
                      Patch(facecolor='gray', label='Not significant')]
    plt.legend(handles=legend_elements)
    
    plt.show()

## 8. Network Visualization

In [ ]:
# Plot the integrated network (simplified version)
try:
    integrator.plot_networks(
        network_type='integrated',
        layout='spring',
        node_size=50,
        edge_width=0.5,
        alpha=0.7
    )
except Exception as e:
    print(f"Network plotting not available in this example: {e}")
    print("In a real implementation, this would show an interactive network plot.")

## 9. Save Results

In [ ]:
# Save all results to files
integrator.save_results(
    output_dir='../results/multiomics_analysis',
    save_networks=True,
    save_metrics=True,
    save_differential=True
)

## 10. Summary and Interpretation

This example demonstrated the complete MultiOmicsNet workflow:

1. **Data Integration**: Successfully integrated scRNA-seq, scATAC-seq, metabolomics, and microbiome data
2. **Network Construction**: Built cross-layer networks using hybrid inference and knowledge-based approaches
3. **Network Quantification**: Computed comprehensive network metrics to characterize network properties
4. **Differential Analysis**: Identified network differences between young and aged conditions
5. **Visualization**: Created informative plots to visualize results

### Key Findings (Example):
- Network topology differs significantly between young and aged conditions
- Specific gene-metabolite and host-microbe interactions are altered with aging
- Hub nodes in the network may represent key regulatory elements

### Next Steps:
1. **Pathway Analysis**: Investigate which biological pathways are most affected
2. **Hub Analysis**: Focus on highly connected nodes that may be key regulators
3. **Validation**: Validate key findings using independent datasets or experimental approaches
4. **Mechanistic Studies**: Design experiments to test causal relationships identified in the networks

In [ ]:
# Print session information
import sys
print("Python version:", sys.version)
print("MultiOmicsNet version:", mon.__version__)
print("Analysis completed successfully!")